# MAG7 Spending Data Analysis
## Comprehensive Summary Statistics and Gap Analysis


In [1]:
import json
import pandas as pd
import numpy as np
from collections import defaultdict, Counter
from datetime import datetime

# Load the data
with open('mag7_spending.json', 'r', encoding='utf-8') as f:
    mag7_data = json.load(f)

print("Data loaded successfully!")
print(f"Companies in dataset: {list(mag7_data.keys())}")


Data loaded successfully!
Companies in dataset: ['Apple', 'Microsoft', 'Alphabet', 'Amazon', 'NVIDIA', 'Tesla', 'Meta']


## 1. Data Structure Overview


In [2]:
# Get all unique domains
all_domains = set()
for company_data in mag7_data.values():
    all_domains.update(company_data.keys())

print(f"Total unique domains: {len(all_domains)}")
print(f"\nDomains: {sorted(all_domains)}")

# Count records per company
record_counts = {}
for company, domains in mag7_data.items():
    total_records = sum(len(records) for records in domains.values())
    record_counts[company] = total_records

print(f"\nTotal records per company:")
for company, count in sorted(record_counts.items(), key=lambda x: x[1], reverse=True):
    print(f"  {company}: {count} records")


Total unique domains: 34

Domains: ['5g_6g_network_infrastructure', 'ai_efficiency_methods', 'ai_foundation_models', 'autonomous_mobility', 'batteries_and_storage', 'bio_ai_and_drug_discovery', 'circularity_reuse_recycling', 'cloud_data_centers_ai_infra', 'content_production_studios', 'digital_health_devices', 'digital_inclusion_and_economy', 'diversity_equity_inclusion', 'edge_computing_infrastructure', 'gaming_platforms_engines', 'industrial_automation_supply_chain', 'litigation_settlements_fines', 'mergers_and_acquisitions', 'quantum_computing', 'r_and_d', 'regulatory_compliance_investments', 'renewable_energy_generation', 'responsible_ai_safety_governance', 'robotics_autonomy', 'satellite_and_subsea_infrastructure', 'sci_fi_consumer_devices', 'security_cryptography', 'semiconductor_ai_hardware', 'share_repurchases', 'strategic_investments', 'superconducting_devices_squids', 'supply_chain_digitization', 'wellness_vr_and_behavioral', 'workforce_training_reskilling', 'xr_metaverse_and

## 2. Total Spending by Company


In [3]:
def calculate_company_spending(company_name):
    """Calculate total documented spending for a company"""
    total = 0
    count_with_amount = 0
    count_null = 0
    
    for domain, records in mag7_data[company_name].items():
        for record in records:
            amount = record.get('investment_amount_billions_usd')
            if amount is not None and isinstance(amount, (int, float)):
                total += amount
                count_with_amount += 1
            else:
                count_null += 1
    
    return {
        'total_billions': round(total, 2),
        'records_with_amount': count_with_amount,
        'records_null_amount': count_null,
        'percent_documented': round(count_with_amount / (count_with_amount + count_null) * 100, 1) if (count_with_amount + count_null) > 0 else 0
    }

company_spending = {}
for company in mag7_data.keys():
    company_spending[company] = calculate_company_spending(company)

# Create DataFrame
df_company_spending = pd.DataFrame(company_spending).T
df_company_spending = df_company_spending.sort_values('total_billions', ascending=False)

print("\n=== TOTAL SPENDING BY COMPANY ===")
print(df_company_spending.to_string())
print(f"\nGrand Total: ${df_company_spending['total_billions'].sum():.2f}B")
print(f"Average documentation rate: {df_company_spending['percent_documented'].mean():.1f}%")



=== TOTAL SPENDING BY COMPANY ===
           total_billions  records_with_amount  records_null_amount  percent_documented
Apple             1820.95                 41.0                 17.0                70.7
Alphabet           602.48                 47.0                 18.0                72.3
Microsoft          513.55                 50.0                 13.0                79.4
Meta               358.61                 35.0                 12.0                74.5
Amazon             259.54                 44.0                 15.0                74.6
NVIDIA             170.26                 38.0                  6.0                86.4
Tesla               32.89                 23.0                  9.0                71.9

Grand Total: $3758.28B
Average documentation rate: 75.7%


## 3. Spending by Domain (Aggregated Across All Companies)


In [4]:
def calculate_domain_spending():
    """Calculate spending by domain across all companies"""
    domain_stats = defaultdict(lambda: {
        'total_billions': 0,
        'record_count': 0,
        'records_with_amount': 0,
        'records_null': 0,
        'companies_active': set()
    })
    
    for company, domains in mag7_data.items():
        for domain, records in domains.items():
            domain_stats[domain]['companies_active'].add(company)
            for record in records:
                domain_stats[domain]['record_count'] += 1
                amount = record.get('investment_amount_billions_usd')
                if amount is not None and isinstance(amount, (int, float)):
                    domain_stats[domain]['total_billions'] += amount
                    domain_stats[domain]['records_with_amount'] += 1
                else:
                    domain_stats[domain]['records_null'] += 1
    
    # Convert to DataFrame
    rows = []
    for domain, stats in domain_stats.items():
        rows.append({
            'domain': domain,
            'total_billions': round(stats['total_billions'], 2),
            'total_records': stats['record_count'],
            'with_amount': stats['records_with_amount'],
            'null_amount': stats['records_null'],
            'companies_count': len(stats['companies_active']),
            'documentation_rate': round(stats['records_with_amount'] / stats['record_count'] * 100, 1) if stats['record_count'] > 0 else 0
        })
    
    return pd.DataFrame(rows)

df_domain_spending = calculate_domain_spending()
df_domain_spending = df_domain_spending.sort_values('total_billions', ascending=False)

print("\n=== TOP 20 DOMAINS BY TOTAL SPENDING ===")
print(df_domain_spending.head(20).to_string(index=False))

print("\n=== DOMAINS WITH LOWEST DOCUMENTATION RATES ===")
print(df_domain_spending.nsmallest(10, 'documentation_rate')[['domain', 'total_records', 'documentation_rate']].to_string(index=False))



=== TOP 20 DOMAINS BY TOTAL SPENDING ===
                             domain  total_billions  total_records  with_amount  null_amount  companies_count  documentation_rate
                  share_repurchases         1717.47             60           60            0                7               100.0
      digital_inclusion_and_economy          988.50             10           10            0                5               100.0
        cloud_data_centers_ai_infra          239.33             36           34            2                7                94.4
 xr_metaverse_and_spatial_computing          168.96             10            9            1                3                90.0
         content_production_studios          159.20              8            7            1                4                87.5
               ai_foundation_models           79.64             25           23            2                6                92.0
                autonomous_mobility           68

## 8. Capex Type & Tech Breakdown


In [5]:

def analyze_capex_breakdown():
    """Analyze capex spending by capex_type and capex_tech for each company."""
    capex_data = defaultdict(lambda: defaultdict(lambda: {'total_billions': 0, 'record_count': 0}))

    for company, domains in mag7_data.items():
        for domain, records in domains.items():
            for record in records:
                if record.get('spend_type') == 'capex':
                    capex_type = record.get('capex_type', 'unspecified_type')
                    capex_tech = record.get('capex_tech', 'unspecified_tech')
                    amount = record.get('investment_amount_billions_usd')

                    if amount is not None and isinstance(amount, (int, float)):
                        key = f"{capex_type} - {capex_tech}"
                        capex_data[company][key]['total_billions'] += amount
                        capex_data[company][key]['record_count'] += 1
    
    # Format for display
    company_capex_summary = {}
    for company, types in capex_data.items():
        rows = []
        for key, stats in types.items():
            rows.append({
                'Capex Type - Tech': key,
                'Total Billions USD': round(stats['total_billions'], 2),
                'Record Count': stats['record_count']
            })
        company_capex_summary[company] = pd.DataFrame(rows).sort_values(by='Total Billions USD', ascending=False)
    
    return company_capex_summary

company_capex_breakdowns = analyze_capex_breakdown()

capex_rollup_rows = []
for company, df_capex in company_capex_breakdowns.items():
    for _, row in df_capex.iterrows():
        type_tech = row['Capex Type - Tech']
        if ' - ' in type_tech:
            capex_type, capex_tech = type_tech.split(' - ', 1)
        else:
            capex_type = type_tech
            capex_tech = 'Other'
        capex_rollup_rows.append({
            'company': company,
            'capex_type': capex_type,
            'capex_tech': capex_tech,
            'total_billions': row['Total Billions USD'],
            'record_count': row['Record Count']
        })

df_capex_rollup = pd.DataFrame(capex_rollup_rows)
df_capex_type_totals = pd.DataFrame()
if not df_capex_rollup.empty:
    df_capex_type_totals = (df_capex_rollup
        .groupby(['company', 'capex_type'], as_index=False)
        .agg({'total_billions': 'sum', 'record_count': 'sum'})
        .sort_values(['company', 'total_billions'], ascending=[True, False]))

print("\n=== CAPEX TYPE & TECH BREAKDOWN (BY COMPANY) ===")

for company, df_capex in company_capex_breakdowns.items():
    if not df_capex.empty:
        print(f"\n{company}:")
        print(df_capex.to_string(index=False))
    else:
        print(f"\n{company}: No capex records found.")




=== CAPEX TYPE & TECH BREAKDOWN (BY COMPANY) ===

Apple:
                   Capex Type - Tech  Total Billions USD  Record Count
 Growth - Manufacturing & Production              500.00             1
     Growth - Digital Infrastructure               10.00             1
                      Growth - Other                4.70             1
       Growth - Corporate Facilities                2.00             2
Maintenance - Digital Infrastructure                0.45             1

Microsoft:
              Capex Type - Tech  Total Billions USD  Record Count
Growth - Digital Infrastructure               24.63             8
                 Growth - Other                0.16             1

Alphabet:
              Capex Type - Tech  Total Billions USD  Record Count
Growth - Digital Infrastructure                23.5             3
                 Growth - Other                 0.6             1

Amazon:
               Capex Type - Tech  Total Billions USD  Record Count
 Growth - Digital Inf

## 9. Opex Type Breakdown

In [6]:

def analyze_opex_breakdown():
    """Analyze opex spending by opex_type for each company."""
    opex_data = defaultdict(lambda: defaultdict(lambda: {'total_billions': 0, 'record_count': 0}))

    for company, domains in mag7_data.items():
        for domain, records in domains.items():
            for record in records:
                if record.get('spend_type') == 'opex':
                    opex_type = record.get('opex_type', 'unspecified_type')
                    amount = record.get('investment_amount_billions_usd')

                    if amount is not None and isinstance(amount, (int, float)):
                        opex_data[company][opex_type]['total_billions'] += amount
                        opex_data[company][opex_type]['record_count'] += 1
    
    # Format for display
    company_opex_summary = {}
    for company, types in opex_data.items():
        rows = []
        for key, stats in types.items():
            rows.append({
                'Opex Type': key,
                'Total Billions USD': round(stats['total_billions'], 2),
                'Record Count': stats['record_count']
            })
        company_opex_summary[company] = pd.DataFrame(rows).sort_values(by='Total Billions USD', ascending=False)
    
    return company_opex_summary

ope_capex_breakdowns = analyze_opex_breakdown()

opex_rollup_rows = []
for company, df_opex in ope_capex_breakdowns.items():
    for _, row in df_opex.iterrows():
        opex_rollup_rows.append({
            'company': company,
            'opex_type': row['Opex Type'],
            'total_billions': row['Total Billions USD'],
            'record_count': row['Record Count']
        })

df_opex_rollup = pd.DataFrame(opex_rollup_rows)
df_opex_type_totals = pd.DataFrame()
if not df_opex_rollup.empty:
    df_opex_type_totals = (df_opex_rollup
        .groupby(['company', 'opex_type'], as_index=False)
        .agg({'total_billions': 'sum', 'record_count': 'sum'})
        .sort_values(['company', 'total_billions'], ascending=[True, False]))

print("\n=== OPEX TYPE BREAKDOWN (BY COMPANY) ===")

for company, df_opex in ope_capex_breakdowns.items():
    if not df_opex.empty:
        print(f"\n{company}:")
        print(df_opex.to_string(index=False))
    else:
        print(f"\n{company}: No opex records found.")




=== OPEX TYPE BREAKDOWN (BY COMPANY) ===

Apple:
       Opex Type  Total Billions USD  Record Count
             R&D               491.2             5
           Other                26.9             3
Legal Settlement                16.5             1

Microsoft:
Opex Type  Total Billions USD  Record Count
      R&D               20.00             1
    Other                1.62             4
 COR/COGS                0.98             1

Alphabet:
       Opex Type  Total Billions USD  Record Count
             R&D               31.00             2
        COR/COGS               30.00             1
Legal Settlement               26.35             6
           Other               10.18             2

Amazon:
Opex Type  Total Billions USD  Record Count
    Other               24.10             3
      S&M                0.15             1
      R&D                0.11             1

NVIDIA:
Opex Type  Total Billions USD  Record Count
      R&D               17.02             3
    Other 

## 4. Spending by Type (Aggregated by Company)


In [7]:
def analyze_spending_types():
    """Analyze spending by spend_type for each company"""
    company_type_spending = defaultdict(lambda: defaultdict(lambda: {'amount': 0, 'count': 0}))
    
    for company, domains in mag7_data.items():
        for domain, records in domains.items():
            for record in records:
                spend_type = record.get('spend_type') or 'unspecified'
                amount = record.get('investment_amount_billions_usd')
                
                company_type_spending[company][spend_type]['count'] += 1
                if amount is not None and isinstance(amount, (int, float)):
                    company_type_spending[company][spend_type]['amount'] += amount
    
    # Create summary DataFrame
    rows = []
    for company, types in company_type_spending.items():
        for spend_type, stats in types.items():
            rows.append({
                'company': company,
                'spend_type': spend_type,
                'total_billions': round(stats['amount'], 2),
                'record_count': stats['count']
            })
    
    return pd.DataFrame(rows)

df_spending_types = analyze_spending_types()

print("\n=== SPENDING BY TYPE (BY COMPANY) ===")
for company in sorted(mag7_data.keys()):
    company_data = df_spending_types[df_spending_types['company'] == company]
    if not company_data.empty:
        print(f"\n{company}:")
        print(company_data[['spend_type', 'total_billions', 'record_count']].sort_values('total_billions', ascending=False).to_string(index=False))

# Aggregate by type across all companies
type_totals = df_spending_types.groupby('spend_type').agg({
    'total_billions': 'sum',
    'record_count': 'sum'
}).sort_values('total_billions', ascending=False)

print("\n=== AGGREGATE SPENDING BY TYPE (ALL COMPANIES) ===")
print(type_totals.to_string())



=== SPENDING BY TYPE (BY COMPANY) ===

Alphabet:
spend_type  total_billions  record_count
    equity          429.93            26
      opex           97.52            14
       M&A           41.73             6
     capex           24.10             7
       ppa            9.20            12

Amazon:
spend_type  total_billions  record_count
     capex          184.05            16
       M&A           28.88             7
      opex           24.36             7
    equity           22.25            18
       ppa            0.00            11

Apple:
spend_type  total_billions  record_count
    equity          763.70            20
      opex          534.60            14
     capex          517.15             7
       M&A            4.90            10
       ppa            0.60             7

Meta:
spend_type  total_billions  record_count
    equity          194.83            13
      opex          120.84            13
     capex           25.84            12
       ppa           14.

## 5. Temporal Analysis (Spending by Year)


In [8]:
def analyze_temporal_trends():
    """Analyze spending trends over time"""
    yearly_data = defaultdict(lambda: {'amount': 0, 'count': 0, 'null_year': 0})

    for company, domains in mag7_data.items():
        for domain, records in domains.items():
            for record in records:
                year = record.get('year')
                amount = record.get('investment_amount_billions_usd')

                if year is not None and isinstance(year, int):
                    yearly_data[year]['count'] += 1
                    if amount is not None and isinstance(amount, (int, float)):
                        yearly_data[year]['amount'] += amount
                else:
                    yearly_data['Unknown']['null_year'] += 1

    # Separate years and "Unknown" to avoid TypeError in sorting
    years = [k for k in yearly_data.keys() if isinstance(k, int)]
    rows = []
    for year in sorted(years):
        stats = yearly_data[year]
        rows.append({
            'year': year,
            'total_billions': round(stats['amount'], 2),
            'record_count': stats['count']
        })

    df = pd.DataFrame(rows)
    null_year_count = yearly_data['Unknown']['null_year']
    return df, null_year_count

df_yearly, null_year_count = analyze_temporal_trends()

print("\n=== SPENDING BY YEAR ===")
print(df_yearly.to_string(index=False))
print(f"\nRecords with null/missing year: {null_year_count}")

if not df_yearly.empty:
    print(f"\nYear range: {df_yearly['year'].min()} - {df_yearly['year'].max()}")
    print(f"Peak spending year: {df_yearly.loc[df_yearly['total_billions'].idxmax(), 'year']} (${df_yearly['total_billions'].max():.2f}B)")
    print(f"Most documented year: {df_yearly.loc[df_yearly['record_count'].idxmax(), 'year']} ({df_yearly['record_count'].max()} records)")



=== SPENDING BY YEAR ===
 year  total_billions  record_count
 2014            2.97             2
 2015           15.30             5
 2016           54.13            10
 2017           70.18            12
 2018          140.68            20
 2019          143.60            26
 2020          218.83            31
 2021          696.57            36
 2022          298.20            26
 2023          436.60            49
 2024          432.91            79
 2025         1142.12            63
 2026            0.00             1

Records with null/missing year: 8

Year range: 2014 - 2026
Peak spending year: 2025 ($1142.12B)
Most documented year: 2024 (79 records)


## 6. Gap Analysis - Missing Data Identification


In [9]:
def identify_gaps():
    """Identify gaps in data coverage"""
    gaps = {
        'companies_missing_domains': defaultdict(list),
        'domains_with_no_amounts': [],
        'domains_with_few_companies': [],
        'high_null_rate_combinations': []
    }
    
    # Check which companies are missing which domains
    for company in mag7_data.keys():
        company_domains = set(mag7_data[company].keys())
        missing = all_domains - company_domains
        if missing:
            gaps['companies_missing_domains'][company] = sorted(missing)
    
    # Check domains with no documented amounts
    for domain in all_domains:
        total_amount = 0
        total_records = 0
        null_count = 0
        companies_with_domain = []
        
        for company, domains in mag7_data.items():
            if domain in domains:
                companies_with_domain.append(company)
                for record in domains[domain]:
                    total_records += 1
                    amount = record.get('investment_amount_billions_usd')
                    if amount is not None and isinstance(amount, (int, float)):
                        total_amount += amount
                    else:
                        null_count += 1
        
        if total_amount == 0 and total_records > 0:
            gaps['domains_with_no_amounts'].append({
                'domain': domain,
                'record_count': total_records,
                'companies': companies_with_domain
            })
        
        if len(companies_with_domain) <= 2 and len(companies_with_domain) > 0:
            gaps['domains_with_few_companies'].append({
                'domain': domain,
                'company_count': len(companies_with_domain),
                'companies': companies_with_domain
            })
        
        # High null rate (>50%)
        if total_records > 0 and (null_count / total_records) > 0.5:
            gaps['high_null_rate_combinations'].append({
                'domain': domain,
                'null_rate': round(null_count / total_records * 100, 1),
                'total_records': total_records
            })
    
    return gaps

gaps = identify_gaps()

print("\n=== GAP ANALYSIS ===")

print("\n1. Companies Missing Domains:")
for company, missing_domains in sorted(gaps['companies_missing_domains'].items()):
    print(f"\n{company} missing {len(missing_domains)} domains:")
    for domain in missing_domains[:5]:
        print(f"  - {domain}")
    if len(missing_domains) > 5:
        print(f"  ... and {len(missing_domains) - 5} more")

print("\n2. Domains with NO Documented Amounts (All Null):")
for item in gaps['domains_with_no_amounts'][:10]:
    print(f"  - {item['domain']}: {item['record_count']} records, companies: {', '.join(item['companies'])}")

print("\n3. Domains with Few Companies (<= 2):")
for item in sorted(gaps['domains_with_few_companies'], key=lambda x: x['company_count'])[:10]:
    print(f"  - {item['domain']}: {item['company_count']} companies ({', '.join(item['companies'])})")

print("\n4. Domain-Company Combinations with High Null Rates (>50%):")
for item in sorted(gaps['high_null_rate_combinations'], key=lambda x: x['null_rate'], reverse=True)[:15]:
    print(f"  - {item['domain']}: {item['null_rate']}% null ({item['total_records']} records)")



=== GAP ANALYSIS ===

1. Companies Missing Domains:

Alphabet missing 12 domains:
  - 5g_6g_network_infrastructure
  - digital_health_devices
  - edge_computing_infrastructure
  - gaming_platforms_engines
  - industrial_automation_supply_chain
  ... and 7 more

Amazon missing 13 domains:
  - 5g_6g_network_infrastructure
  - bio_ai_and_drug_discovery
  - digital_inclusion_and_economy
  - gaming_platforms_engines
  - litigation_settlements_fines
  ... and 8 more

Apple missing 9 domains:
  - bio_ai_and_drug_discovery
  - edge_computing_infrastructure
  - industrial_automation_supply_chain
  - quantum_computing
  - regulatory_compliance_investments
  ... and 4 more

Meta missing 16 domains:
  - 5g_6g_network_infrastructure
  - autonomous_mobility
  - batteries_and_storage
  - bio_ai_and_drug_discovery
  - content_production_studios
  ... and 11 more

Microsoft missing 14 domains:
  - bio_ai_and_drug_discovery
  - edge_computing_infrastructure
  - industrial_automation_supply_chain
  - li

## 10. Domain Coverage Matrix


In [10]:
def create_coverage_matrix():
    """Create a matrix showing which companies have data in which domains"""
    matrix_data = []
    
    for domain in sorted(all_domains):
        row = {'domain': domain}
        for company in sorted(mag7_data.keys()):
            if domain in mag7_data[company]:
                record_count = len(mag7_data[company][domain])
                # Count how many have amounts
                with_amount = sum(1 for r in mag7_data[company][domain] 
                                if r.get('investment_amount_billions_usd') is not None)
                row[company] = f"{with_amount}/{record_count}"
            else:
                row[company] = "0/0"
        matrix_data.append(row)
    
    return pd.DataFrame(matrix_data)

df_coverage = create_coverage_matrix()

print("\n=== DOMAIN COVERAGE MATRIX ===")
print("Format: (records_with_amount / total_records)\n")
print(df_coverage.to_string(index=False))



=== DOMAIN COVERAGE MATRIX ===
Format: (records_with_amount / total_records)

                             domain Alphabet Amazon Apple  Meta Microsoft NVIDIA Tesla
       5g_6g_network_infrastructure      0/0    0/0   1/1   0/0       1/1    0/0   0/0
              ai_efficiency_methods      1/2    1/1   0/2   0/1       0/1    1/2   2/2
               ai_foundation_models      1/1    2/2   1/2   1/2       9/9    9/9   0/0
                autonomous_mobility      4/4    1/1   2/2   0/0       1/1    2/2   3/3
              batteries_and_storage      1/2    2/2   0/1   0/0       1/1    0/0   7/8
          bio_ai_and_drug_discovery      1/1    0/0   0/0   0/0       0/0    2/2   0/0
        circularity_reuse_recycling      0/2    2/2   0/1   0/2       1/1    2/2   0/1
        cloud_data_centers_ai_infra      5/6    7/7   2/2   3/3       7/7    8/9   2/2
         content_production_studios      1/1    2/2   3/4   0/0       1/1    0/0   0/0
             digital_health_devices      0/0    1/1

## 11. Research Priority Recommendations


In [11]:
def generate_research_priorities():
    """Generate prioritized list of areas needing more research"""
    priorities = []
    
    # Priority 1: Domains with high activity but low documentation
    for _, row in df_domain_spending.iterrows():
        if row['total_records'] >= 5 and row['documentation_rate'] < 50:
            priorities.append({
                'priority': 'HIGH',
                'reason': 'High activity, low documentation',
                'domain': row['domain'],
                'records': row['total_records'],
                'doc_rate': row['documentation_rate'],
                'action': f"Find dollar amounts for {row['null_amount']} records"
            })
    
    # Priority 2: Major companies missing entire domains
    for company, missing in gaps['companies_missing_domains'].items():
        if len(missing) > 10:
            priorities.append({
                'priority': 'MEDIUM',
                'reason': 'Company missing many domains',
                'domain': f"Multiple ({len(missing)} domains)",
                'company': company,
                'action': f"Research {company}'s activity in: {', '.join(list(missing)[:3])}..."
            })
    
    # Priority 3: Domains with only 1-2 companies (potential gaps)
    for item in gaps['domains_with_few_companies']:
        if item['company_count'] == 1:
            missing_companies = set(mag7_data.keys()) - set(item['companies'])
            priorities.append({
                'priority': 'LOW',
                'reason': 'Only 1 company in domain',
                'domain': item['domain'],
                'action': f"Check if {', '.join(list(missing_companies)[:3])} have activity here"
            })
    
    return pd.DataFrame(priorities)

df_priorities = generate_research_priorities()

print("\n=== RESEARCH PRIORITY RECOMMENDATIONS ===")
print("\nHIGH PRIORITY (High activity domains with low documentation):")
high_pri = df_priorities[df_priorities['priority'] == 'HIGH']
if not high_pri.empty:
    print(high_pri[['domain', 'records', 'doc_rate', 'action']].to_string(index=False))

print("\nMEDIUM PRIORITY (Companies missing many domains):")
med_pri = df_priorities[df_priorities['priority'] == 'MEDIUM']
if not med_pri.empty:
    print(med_pri[['company', 'domain', 'action']].head(10).to_string(index=False))

print("\nLOW PRIORITY (Single-company domains):")
low_pri = df_priorities[df_priorities['priority'] == 'LOW']
if not low_pri.empty:
    print(low_pri[['domain', 'action']].head(10).to_string(index=False))



=== RESEARCH PRIORITY RECOMMENDATIONS ===

HIGH PRIORITY (High activity domains with low documentation):
                     domain  records  doc_rate                             action
renewable_energy_generation     59.0      20.3 Find dollar amounts for 47 records
      security_cryptography      7.0      42.9  Find dollar amounts for 4 records
      ai_efficiency_methods     11.0      45.5  Find dollar amounts for 6 records
circularity_reuse_recycling     11.0      45.5  Find dollar amounts for 6 records

MEDIUM PRIORITY (Companies missing many domains):
  company                domain                                                                                                                            action
Microsoft Multiple (14 domains) Research Microsoft's activity in: bio_ai_and_drug_discovery, edge_computing_infrastructure, industrial_automation_supply_chain...
 Alphabet Multiple (12 domains)           Research Alphabet's activity in: 5g_6g_network_infrastructure, digi

## 12. Save All Results to File


In [12]:
# Create comprehensive summary report
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
output_file = f"mag7_analysis_summary_{timestamp}.txt"

with open(output_file, 'w', encoding='utf-8') as f:
    f.write("="*80 + "\n")
    f.write("MAG7 SPENDING DATA - COMPREHENSIVE ANALYSIS SUMMARY\n")
    f.write(f"Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
    f.write("="*80 + "\n\n")
    
    f.write("EXECUTIVE SUMMARY\n")
    f.write("-" * 80 + "\n")
    f.write(f"Total Companies: {len(mag7_data)}\n")
    f.write(f"Total Domains: {len(all_domains)}\n")
    f.write(f"Total Records: {sum(record_counts.values())}\n")
    f.write(f"Total Documented Spending: ${df_company_spending['total_billions'].sum():.2f}B\n")
    f.write(f"Average Documentation Rate: {df_company_spending['percent_documented'].mean():.1f}%\n")
    f.write("\n\n")
    
    f.write("1. SPENDING BY COMPANY\n")
    f.write("-" * 80 + "\n")
    f.write(df_company_spending.to_string())
    f.write("\n\n")
    
    f.write("2. TOP 20 DOMAINS BY SPENDING\n")
    f.write("-" * 80 + "\n")
    f.write(df_domain_spending.head(20).to_string(index=False))
    f.write("\n\n")
    
    f.write("3. SPENDING BY TYPE (AGGREGATE)\n")
    f.write("-" * 80 + "\n")
    f.write(type_totals.to_string())
    f.write("\n\n")
    
    f.write("4. SPENDING BY YEAR\n")
    f.write("-" * 80 + "\n")
    f.write(df_yearly.to_string(index=False))
    f.write(f"\nRecords with missing year: {null_year_count}\n")
    f.write("\n\n")
    
    f.write("5. DOMAIN COVERAGE MATRIX\n")
    f.write("-" * 80 + "\n")
    f.write("Format: (records_with_amount / total_records)\n\n")
    f.write(df_coverage.to_string(index=False))
    f.write("\n\n")
    
    f.write("6. GAP ANALYSIS - MISSING DATA IDENTIFICATION\n")
    f.write("-" * 80 + "\n")
    for item in gaps['domains_with_no_amounts']:
        f.write(f"  - {item['domain']}: {item['record_count']} records\n")
    f.write("\n\n")
    
    f.write("7. CAPEX TYPE & TECH BREAKDOWN (BY COMPANY)\n")
    f.write("-" * 80 + "\n")
    for company, df_capex in company_capex_breakdowns.items():
        if not df_capex.empty:
            f.write(f"\n{company}:\n")
            f.write(df_capex.to_string(index=False) + "\n")
        else:
            f.write(f"\n{company}: No capex records found.\n")
    f.write("\n\n")
    
    f.write("8. CAPEX TYPE TOTALS (ROLLUPS)\n")
    f.write("-" * 80 + "\n")
    if not df_capex_type_totals.empty:
        f.write(df_capex_type_totals.to_string(index=False))
    else:
        f.write("No aggregated capex totals available.\n")
    f.write("\n\n")
    
    f.write("9. OPEX TYPE BREAKDOWN (BY COMPANY)\n")
    f.write("-" * 80 + "\n")
    for company, df_opex in ope_capex_breakdowns.items():
        if not df_opex.empty:
            f.write(f"\n{company}:\n")
            f.write(df_opex.to_string(index=False) + "\n")
        else:
            f.write(f"\n{company}: No opex records found.\n")
    f.write("\n\n")
    
    f.write("10. OPEX TYPE TOTALS\n")
    f.write("-" * 80 + "\n")
    if not df_opex_type_totals.empty:
        f.write(df_opex_type_totals.to_string(index=False))
    else:
        f.write("No aggregated opex totals available.\n")
    f.write("\n\n")
    
    f.write("11. RESEARCH PRIORITY RECOMMENDATIONS\n")
    f.write("-" * 80 + "\n")
    if not df_priorities.empty:
        f.write(df_priorities.to_string(index=False) + "\n")
    else:
        f.write("No specific research priorities identified based on current criteria.\n")
    f.write("\n\n")

print(f"\n✓ Analysis complete! Results saved to: {output_file}")



✓ Analysis complete! Results saved to: mag7_analysis_summary_20251202_165728.txt
